In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

poet_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "프로그래밍 언어의 문법과 구조에서 영감을 받아, 비유적 표현을 사용해 시를 써보세요."
        ),
        (
            "human",
            "{language}를 주제로 시를 만들어줘."
        ),
    ]
)

poet_chain = poet_prompt | chat

critic_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 유명한 문학 평론가입니다. 주어진 시를 해석하고 이를 쉬운 표현으로 설명해주세요."
        ),
        (
            "human",
            "{poem}"
        ),
    ]
)

critic_chain = critic_prompt | chat

final_chain = {"poem" : poet_chain} | critic_chain
final_chain.invoke({"language": "python"})

코드처럼 흘러가는 파이썬의 노래
들려오는 듯한 코딩의 소리
들여다보면 무한한 세계가 펼쳐져
변수와 함수가 춤을 추는 파티

들여다보면 논리의 미로
들여다보면 알고리즘의 향연
한 줄 한 줄 읽어가며
문제를 해결하는 즐거움

파이썬의 힘을 느껴보면
세상은 더 넓고 아름다워지고
코드 한 줄 한 줄이
작은 기적을 만들어내는 것 같네.이 시는 파이썬이라는 프로그래밍 언어에 대한 찬사를 담고 있습니다. 파이썬은 마치 코드처럼 흘러가며 아름다운 노래를 부르는 것처럼 느껴지고, 코딩을 하면서 무한한 세계를 만나게 됩니다. 변수와 함수가 춤을 추는 파티처럼, 파이썬은 코딩을 즐겁게 만들어줍니다. 논리의 미로와 알고리즘의 향연 속에서 문제를 해결하는 즐거움을 느낄 수 있으며, 파이썬의 힘을 느끼면 세상이 더 아름다워지고 작은 기적을 만들어내는 기분을 느낄 수 있습니다.

AIMessageChunk(content='이 시는 파이썬이라는 프로그래밍 언어에 대한 찬사를 담고 있습니다. 파이썬은 마치 코드처럼 흘러가며 아름다운 노래를 부르는 것처럼 느껴지고, 코딩을 하면서 무한한 세계를 만나게 됩니다. 변수와 함수가 춤을 추는 파티처럼, 파이썬은 코딩을 즐겁게 만들어줍니다. 논리의 미로와 알고리즘의 향연 속에서 문제를 해결하는 즐거움을 느낄 수 있으며, 파이썬의 힘을 느끼면 세상이 더 아름다워지고 작은 기적을 만들어내는 기분을 느낄 수 있습니다.')